In [1]:
import pyrealsense2 as rs
import numpy as np
from enum import IntEnum

from datetime import datetime
import open3d as o3d

def get_intrinsic_matrix(frame):
    intrinsics = frame.profile.as_video_stream_profile().intrinsics
    out = o3d.camera.PinholeCameraIntrinsic(640, 480, intrinsics.fx,
                                            intrinsics.fy, intrinsics.ppx,
                                            intrinsics.ppy)
    return out

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()

#color_profiles, depth_profiles = get_profiles()
#print('Using the default profiles: \n  color:{}, depth:{}'.format(
#	     color_profiles[0], depth_profiles[0]))

#w, h, fps, fmt = depth_profiles[0]
#Depth profile 1280 720 30 format.z16
#Colour profile 1920 1080 30 format.rgb8
w = 1280
h = 720
fps = 30
fmt = rs.format.z16
config.enable_stream(rs.stream.depth, w, h, fmt, fps)
#w, h, fps, fmt = color_profiles[0]
#Colour profile 1920 1080 30 format.rgb8
w = 1920
h = 1080
fps = 30
fmt = rs.format.rgb8
config.enable_stream(rs.stream.color, w, h, fmt, fps)

# Start streaming
profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()

# Using preset HighAccuracy for recording
#depth_sensor.set_option(rs.option.visual_preset, o3d.Preset.HighAccuracy)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_scale = depth_sensor.get_depth_scale()


# We will not display the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 3  # 3 meter
clipping_distance = clipping_distance_in_meters / depth_scale
# print(depth_scale)


# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

vis = o3d.visualization.Visualizer()
vis.create_window()


pcd = o3d.geometry.PointCloud()
flip_transform = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]

# Streaming loop
frame_count = 0

try:
    for fid in range(100):

        dt0 = datetime.now()

        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        intrinsic = o3d.camera.PinholeCameraIntrinsic(
            get_intrinsic_matrix(color_frame))

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = o3d.geometry.Image(
            np.array(aligned_depth_frame.get_data()))
        color_temp = np.asarray(color_frame.get_data())
        color_image = o3d.geometry.Image(color_temp)

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
            color_image,
            depth_image,
            depth_scale=1.0 / depth_scale,
            depth_trunc=clipping_distance_in_meters,
            convert_rgb_to_intensity=False)
        temp = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image, intrinsic)
        temp.transform(flip_transform)

        pcd.points = temp.points
        pcd.colors = temp.colors
        pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

        oboxes = pcd.detect_planar_patches(
            normal_variance_threshold_deg=60,
            coplanarity_deg=75,
            outlier_ratio=0.75,
            min_plane_edge_length=0,
            min_num_points=0,
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

        print("Detected {} patches".format(len(oboxes)))

        geometries = []
        for obox in oboxes:
            mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
            mesh.paint_uniform_color(obox.color)
            geometries.append(mesh)
            geometries.append(obox)
        geometries.append(pcd)

        o3d.visualization.draw_geometries(geometries,
                                      zoom=0.62,
                                      front=[0.4361, -0.2632, -0.8605],
                                      lookat=[2.4947, 1.7728, 1.5541],
                                      up=[-0.1726, -0.9630, 0.2071])
        process_time = datetime.now() - dt0
        print("FPS: " + str(1 / process_time.total_seconds()))

        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        

finally:
    print("Stop pipeline")
    pipeline.stop()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Detected 23 patches
FPS: 8.243341155755751e-05
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Detected 24 patches
FPS: 0.07832184482406172
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Er

RuntimeError: stop() cannot be called before start()

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                        ransac_n=3,
                                        num_iterations=1000)


In [ ]:
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

print(f"Inliers {len(inliers)}")

In [ ]:
print(f"All {pcd}")

In [ ]:
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

oboxes = pcd.detect_planar_patches(
    normal_variance_threshold_deg=60,
    coplanarity_deg=75,
    outlier_ratio=0.75,
    min_plane_edge_length=0,
    min_num_points=0,
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

print("Detected {} patches".format(len(oboxes)))

geometries = []
for obox in oboxes:
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    geometries.append(mesh)
    geometries.append(obox)
geometries.append(pcd)

o3d.visualization.draw_geometries(geometries,
                                  zoom=0.62,
                                  front=[0.4361, -0.2632, -0.8605],
                                  lookat=[2.4947, 1.7728, 1.5541],
                                  up=[-0.1726, -0.9630, 0.2071])

In [ ]:
while True:
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()